<a href="https://colab.research.google.com/github/jyotidabass/Medical_test_classification/blob/main/Classify_MedicalText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import re
from nltk.corpus import stopwords
from string import digits
#nltk.download("wordnet")
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report


In [93]:
!git clone https://github.com/SnehaVM/Medical-Text-Classification--MachineLearning

fatal: destination path 'Medical-Text-Classification--MachineLearning' already exists and is not an empty directory.


In [53]:
# open docs file and read its lines
with open("/content/Medical-Text-Classification--MachineLearning/data/Medical.dat", "r") as fh:
    lines = fh.readlines() 
    
train_arr = []
train_lbl = []

for i in range(len(lines)):
    train_arr.append(lines[i].lower()[2:])
    train_lbl.append(lines[i][0:1])

print (train_arr[0:2])
print (train_lbl[0:2])

['"diabetes mellitus refers to a group of diseases that affect how your body uses blood sugar (glucose). glucose is vital to your health because it\'s an important source of energy for the cells that make up your muscles and tissues. it\'s also your brain\'s main source of fuel. the underlying cause of diabetes varies by type. but, no matter what type of diabetes you have, it can lead to excess sugar in your blood. too much sugar in your blood can lead to serious health problems. chronic diabetes conditions include type 1 diabetes and type 2 diabetes. potentially reversible diabetes conditions include prediabetes and gestational diabetes. prediabetes occurs when your blood sugar levels are higher than normal, but not high enough to be classified as diabetes. and prediabetes is often the precursor of diabetes unless appropriate measures are taken to prevent progression. gestational diabetes occurs during pregnancy but may resolve after the baby is delivered.\n', 'high blood pressure (hy

# open docs file and read its lines
with open("/content/Medical-Text-Classification--MachineLearning/data/Medicaltrain.csv", "r") as fh:lines = fh.readlines() 
    
train_arr = []
train_lbl = []

for i in range(len(lines)):
    train_arr.append(lines[i].lower()[2:])
    train_lbl.append(lines[i][0:1])

print (train_arr[0:2])
print (train_lbl[0:2])

In [54]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
#remove short words
!pip install stopwords
def filterLen(tdocs, minlen):
    return [ ' '.join(t for t in d if len(t) >= minlen ) for d in tdocs ]

trainDocs = [l.split() for l in train_arr]
train_filtered = filterLen(trainDocs,4)

#lemmatize and remove stop words
def preProcess(data_arr):
    lemmatiser = WordNetLemmatizer()
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    #print train_arr[0:10]
    for i in range(len(train_arr)):
        data_arr[i] = pattern.sub('', data_arr[i])
        map = str.maketrans('', '', 'digits')
        data_arr[i] = data_arr[i].translate(map)
        #data_arr[i] = nltk.word_tokenize(lemmatiser.lemmatize(data_arr[i],pos="v").translate(None, string.punctuation))
    return data_arr

train_arr = preProcess(train_filtered)
print (train_arr[0:2])

['"abee mellu refer roup eae affec boy ue bloo uar (lucoe). lucoe val healh \'mporan ource enery cell make mucle ue. \'alo bran\'man ource fuel. unerlyn caue abee vare ype. , maer ype abee , lea exce uar bloo. much uar bloo lea erou healh problem. chronc abee conon nclue ype abee ype abee. poenally reverble abee conon nclue preabee eaonal abee. preabee occur bloo uar level hher normal, hh enouh clafe abee. preabee ofen precuror abee unle approprae meaure aken preven proreon. eaonal abee occur prenancy reolve baby elvere.', 'hh bloo preure (hyperenon) common conon lon-erm force bloo arery wall hh enouh evenually caue healh problem, hear eae. bloo preure eermne amoun bloo hear pump amoun reance bloo flow arere. bloo hear pump narrower arere, hher bloo preure. bloo preure rean ven mllmeer mercury h).']


In [56]:
#create TDIF matrix
vectorizer = CountVectorizer()
tfidf = TfidfTransformer()
vectorizer.fit(train_arr)
x_train_arr = vectorizer.transform(train_arr)
tfidf.fit(x_train_arr)
x_train = tfidf.transform(x_train_arr)
print (x_train.shape)


(140, 126)


In [87]:
# open docs file and read its lines - for test mode
with open("/content/Medical-Text-Classification--MachineLearning/data/Medicaltest.dat", "r") as fh: linesTest = fh.readlines() 
for i in range(len(linesTest)):
 test_arr.append(linesTest[i].lower())
  
# test_arr = preProcess(test_arr)
vectorizer.fit(test_arr)
x_test_arr = vectorizer.transform(test_arr)
tfidf.fit(x_test_arr)
x_test = tfidf.transform(x_test_arr)

In [88]:
#split to train and test
train_lbl = np.asarray(train_lbl)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_train, train_lbl, test_size=0.25, random_state=42)
print (X_train.shape)
print (X_test.shape)

print (Y_train.shape)
print (Y_test.shape)


(105, 126)
(35, 126)
(105,)
(35,)


In [89]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif,f_classif,VarianceThreshold,chi2,SelectPercentile,SelectFdr
featureSelector1 = SelectKBest(f_classif, k=5)
X_new = featureSelector1.fit_transform(X_train,Y_train)
print (X_new.shape)

X_test_new = featureSelector1.transform(X_test)
print (X_test_new.shape)

X_new = X_train
X_test_new = X_test

(105, 5)
(35, 5)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [91]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
#from sklearn.neural_network import MLPClassifier
kfold = model_selection.StratifiedKFold(n_splits=5)
#model = ExtraTreesClassifier(random_state=42, class_weight ='balanced_subsample', n_estimators=200,min_samples_split = 5)
model = SGDClassifier(loss='hinge', penalty='l2',alpha=0.0004, max_iter=100, random_state=42)
#model = LogisticRegression(class_weight='balanced', C=0.1)
#model = MLPClassifier(max_iter=18)
#model = AdaBoostClassifier(LinearSVC(penalty='l1', dual=False,tol=1e-3, class_weight='balanced',C=0.1),algorithm='SAMME')

model.fit(X_new,Y_train)
predict = model.predict(X_test_new)
#Cross Validate
#results = model_selection.cross_val_score(model, x_train, train_lbl, cv=kfold, scoring='f1_micro')
#print(results.mean())

print ('\n clasification report:\n')
classification_report(Y_test,predict)


 clasification report:



'              precision    recall  f1-score   support\n\n           1       1.00      1.00      1.00         3\n           2       1.00      1.00      1.00         7\n           3       1.00      1.00      1.00         6\n           4       1.00      1.00      1.00         6\n           5       1.00      1.00      1.00        13\n\n    accuracy                           1.00        35\n   macro avg       1.00      1.00      1.00        35\nweighted avg       1.00      1.00      1.00        35\n'